In [226]:
import numpy as np
import pandas as pd
import gensim
import pymorphy2
import tqdm

In [2]:
model = gensim.models.KeyedVectors.load_word2vec_format("180/model.bin", binary=True)

In [10]:
model.init_sims(replace=True)

morph = pymorphy2.MorphAnalyzer()

In [151]:
model.most_similar(["стиральная_ADJ"],topn=15)

KeyError: "word 'стиральная_ADJ' not in vocabulary"

In [198]:
morph.parse("слегка")[0].tag.case

In [227]:
# pymorphy2: word2vec 
cotags = {'ADJF':'ADJ', # прилогательное
'ADJS' : 'ADJ',  # прилогательное
'ADVB' : 'ADV',  # наречие
'COMP' : 'ADV',  # компаратив
'GRND' : 'VERB', # деепричастие
'INFN' : 'VERB', # глагол (инфинитив)
'NOUN' : 'NOUN', # имя существительное
'PRED' : 'ADV',  # предикатив
'PRTF' : 'ADJ',  # причастие
'PRTS' : 'VERB', # причастие
'NPRO' : 'NOUN', # местоимение-существительное
'CONJ' : 'ADV',   
'NUMR' : 'NUM', # числительные
'INTJ' : 'INTJ',
'VERB' : 'VERB'} # глагол


def search_neighbours(word, max_nei=5, topn=20, min_score=0.4):
    result = set([word])
    m = morph.parse(word)[0]
    #gend = m.tag.gender
    pos = m.tag.POS
    word = m.normal_form
    word = word.replace('ё', 'е')
    #print(word, pos, gend)
    if pos not in cotags.keys():
        #print("Not good pos {} ".format(pos))
        return result
    lex = word + '_' + cotags[pos]
    if lex not in model :
        #print("Word {} not in model".format(lex))
        return result

    neighbs = model.most_similar([lex], topn=topn)
    for nei in neighbs:
        #print(nei)
        if nei[1] < min_score:
            break
        lex_n, ps_n = nei[0].split('_')
        #print(lex_n, ps_n)
        if '::' in lex_n or ps_n not in cotags.values() or lex_n.startswith("-"): 
            continue
        if cotags[pos] == ps_n:
            #if True or pos == 'NOUN':
            parse_result = morph.parse(lex_n)
            for ana in parse_result:
                #if ana.normal_form == lex_n:
                if ana.tag.gender == m.tag.gender and \
                ana.tag.number == m.tag.number and \
                ana.tag.case == m.tag.case:
                    #print("сущь")
                    result.add(lex_n)
            #elif cotags[pos] == 'VERB' and word[-2:] == 'ся':
            #    if lex_n[-2:] == 'ся':
            #        print("гл")
            #        result.add(lex_n)
            #elif cotags[pos] == 'VERB' and word[-2:] != 'ся':
            #    if lex_n[-2:] != 'ся':
            #        print("гл")
            #        result.add(lex_n)
            #else:
            #    result.add(lex_n)
        if len(result) >= max_nei:
            break
    return result

In [228]:
for word in "сделай чуть тише".split():
    print("Orig:{}".format(word))
    nei = search_neighbours(word)
    if len(nei) > 0 :
        print("Replace:{}".format(' '.join(nei)))

Orig:сделай
Replace:принесть сделай
Orig:чуть
Replace:немного чуть немножко слегка заметно
Orig:тише
Replace:тише


In [215]:
iot_commands_v01 = pd.read_csv("data/iot_commands_v0.1.csv", sep=";")

print("Total {} rows".format(len(iot_commands_v01)))
variable_text = []
for i, row in iot_commands_v01.iterrows():
    if i % 50 == 0:
        print(i)
    variable_transciptions = [search_neighbours(word, max_nei=10, topn=40, min_score=0.3) for word in row.text.split()]
    #print(variable_transciptions)
    #print("Orig: {}".format(row.text))
    out_str = "[{}]".format("] [".join(["|".join(s) for s in variable_transciptions]))
    variable_text.append(out_str)
iot_commands_v01["variable_text"] = variable_text

In [240]:
iot_commands_v01.head(5)

,type,device,condition,location,params,if_schedule,text,variable_text
0,1,1,0,0,0,1,включай стиральную машину в время каждое четно...,[включай] [стиральную] [опель|машину] [в] [оле...
1,1,1,1,0,0,1,когда стиральная машина закончит стирку сообщи...,[рано|как|тогда|прежде|когда|уже|еще|теперь|ко...
2,1,0,0,0,0,0,включи стирку а потом отжим,[включи|включять] [стирку] [а] [потом|тут|заче...
3,3,0,0,0,0,1,сколько раз за месяц я стирал,[беда|сколько|сколь|много|столько|столько-то|д...
4,1,0,1,0,0,1,имитируй присутствие людей с число июня до чис...,[имитируй] [действие|расположение|нерасположен...


In [244]:
iot_commands_v01.to_csv("data/iot_commands_w2vtext_v0.1.csv", index_label="id", sep=";")